In [10]:
"""
Summarizing Sentinel-2 spectral metrics in gridcells
Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success


In [6]:
# Load the exported asset from above
s2 = ee.Image('projects/jfsp-aspen/assets/afd_aspen_fires_S2metrics_mosaic')
print(s2.bandNames().getInfo())

['MNDWI', 'LAI', 'EVI']


In [7]:
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_aspenfires_gridstats')
print(grid.first().propertyNames().getInfo())

['Fire_Year', 'grid_index', 'Fire_ID', 'max_date', 'afd_count', 'Ig_Date', 'first_date', 'last_date', 'Last_Dat_1', 'system:index']


In [9]:
# Simplify the feature collection properties.
grid = grid.select(['grid_index'])

# define a custom reducer
mean_stdDev = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(), sharedInputs=True)

# Perform the reduction by AFD observations
s2_stats = s2.reduceRegions(
    collection=grid,
    reducer=mean_stdDev,
    scale=30
)

print("Submitted !")

Submitted !


In [11]:
# Export the table. 
s2_stats = s2_stats.map(lambda feature: feature.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=s2_stats,
    description='viirs_snpp_jpss1_afd_aspenfires_gridstats_S2metrics',
    fileNamePrefix='viirs_snpp_jpss1_afd_aspenfires_gridstats_S2metrics',
    fileFormat='CSV', 
    folder='S2'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
monitor_export(export_task) 

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!
